# 1. Fuentes de Datos

In [30]:


import pandas as pd

# PASO 1 CARGAR LOS DATOS

#Definimos las columnas qur vamos a usar el las tablas : Falta explicacion
a_cols = ['anime_id','name']
r_cols =['user_id','anime_id','rating']

#Cargamos los Dataframes y les aplicamos formato
anime_df = pd.read_csv("../data/anime.csv",usecols=a_cols)
rating_df = pd.read_csv("../data/rating.csv",sep=',',usecols=r_cols)

#Vemos las 5 primeras entradas de las dos tablas
print(anime_df.head())
print(rating_df.head())


#Vemos la informacion de las tablas 
anime_df.info()
rating_df.info()    



   anime_id                              name
0     32281                    Kimi no Na wa.
1      5114  Fullmetal Alchemist: Brotherhood
2     28977                          Gintama°
3      9253                       Steins;Gate
4      9969                     Gintama&#039;
   user_id  anime_id  rating
0        1        20      -1
1        1        24      -1
2        1        79      -1
3        1       226      -1
4        1       241      -1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   anime_id  12294 non-null  int64 
 1   name      12294 non-null  object
dtypes: int64(1), object(1)
memory usage: 192.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memo

In [31]:

# PASO 2 FILTRADO DE DATOS
# Quitamos los ratings invalidos, es decir, los que no tienen una evaluacion como tal 
ratings_filtrados = rating_df[rating_df['rating'] != -1]
#print(ratings_filtrados.head())


#FILTRADO DE DATOS 
#Los animes que nos interesan del ejercicio son los que poseen más de 100 calificaciones
#Por ende primero necesitamos saber cuantas calificaciones tiene cada anime
# Creamos una variable y de los ratings filtrados guardamos cuantos ratings tienen
#cada anime agrupado por su id y contando su rating
anime_counts = ratings_filtrados.groupby('anime_id')['rating'].count()
#print(anime_counts) 

#Necesitamos animes más populares con más de 100 calificaciones. 
#Usamos los datos obtenidos recientemente
animes_populares = anime_counts[anime_counts > 100].index # index es el id del dataframe 
#Ahora necesitamos usar estos animes populares para filtrar los ratings
# No nos sirve de nada tener ratings de animes que no sean estos animes populares
# con más de 100 reviews asi que los eliminamos 
rating_df = ratings_filtrados[ratings_filtrados['anime_id'].isin(animes_populares)]

# Lo mismo con los usuarios que no tengan mas de 5 ratings
# Nos interesa quedarnos con usuarios que hayan calificado al menos 5 animes
counts_user = rating_df['user_id'].value_counts() # Contamos cuantos ratings tiene cada usuario
# Filtramos el dataframe de ratings para quedarnos solo con los usuarios que tengan 5 o más ratings
rating_df = rating_df[rating_df['user_id'].isin(counts_user[counts_user >= 5].index)]       


print(rating_df.head())

     user_id  anime_id  rating
156        3        20       8
157        3       154       6
158        3       170       9
159        3       199      10
160        3       225       9


In [32]:
# PASO 3 ELIMINAR DUPLICADOS 
#Identifica filas duplicadas en un DataFrame de ratings (evaluaciones) basándose en las columnas user_id y anime_id,
#marcando como True los duplicados excepto la última ocurrencia.
duplicates_ratings = rating_df.duplicated(subset=['user_id','anime_id'],keep='last')

# Eliminar duplicados directamente, conservando la última ocurrencia
cleaned_df = rating_df.drop_duplicates(subset=['user_id','anime_id'], keep='last')


# 3. Verificar resultado
print(f"Filas originales: {len(rating_df)}")
print(f"Filas después de limpiar: {len(cleaned_df)}")





Filas originales: 6179379
Filas después de limpiar: 6179372


In [33]:

# PASO 4 LECTURA E INTERPRETACION DE RESULTADOS ESTADÍSTICOS    


# Analizamos la distribución de la cantidad de calificaciones por anime
counts_per_anime = rating_df['anime_id'].value_counts()
print(counts_per_anime.describe())
# Calcular e imprimir percentiles específicos 
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_anime.quantile(p/100)))

# Analizamos la distribución de la cantidad de calificaciones por usuario
counts_per_user = rating_df['user_id'].value_counts()
print(counts_per_user.describe())
# Calcular e imprimir percentiles específicos
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_user.quantile(p/100)))


count     4129.000000
mean      1496.580044
std       2528.268554
min        100.000000
25%        230.000000
50%        569.000000
75%       1591.000000
max      33454.000000
Name: count, dtype: float64
P50 = 569
P75 = 1591
P90 = 3769
P95 = 6108
P99 = 12414
count    60919.000000
mean       101.435989
std        132.795031
min          5.000000
25%         22.000000
50%         56.000000
75%        128.000000
max       2781.000000
Name: count, dtype: float64
P50 = 56
P75 = 128
P90 = 244
P95 = 346
P99 = 647


In [36]:
# PASO 5 Identificacion de Outliers
# Un outlier es un dato que se encuentra muy alejado del resto de los datos en un conjunto.
# Tenemos que eliminarlos para evitar que afecten negativamente a nuestro modelo de recomendacion.
MAX_RATINGS_USER = counts_per_user.quantile(0.85) # Definimos un umbral para usuarios con demasiadas calificaciones (outliers)
# Filtramos los usuarios que tienen entre 5 y MAX_RATINGS_USER calificaciones
valid_users = counts_user[
    (counts_user >= 5) &
    (counts_user <= MAX_RATINGS_USER)
].index

MAX_RATINGS_ANIME = counts_per_anime.quantile(0.85) # Definimos un umbral para animes con demasiadas calificaciones (outliers)
# Filtramos los animes que tienen entre 100 y MAX_RATINGS_ANIME calificaciones
valid_animes = counts_per_anime[
    (counts_per_anime >= 100) &
    (counts_per_anime <= MAX_RATINGS_ANIME)
].index


# Filtramos el DataFrame de ratings para quedarnos solo con los usuarios y animes válidos   
rating_df = rating_df[
    rating_df['user_id'].isin(valid_users) &
    rating_df['anime_id'].isin(valid_animes)
]
print(rating_df.head())


      user_id  anime_id  rating
1284       12       302       9
2254       19      8142      10
2265       20       819      10
2269       20      5391       9
2274       20      7739      10


In [ ]:
# PASO 6 VOLVEMOS A LEER E INTERPRETAR RESULTADOS ESTADÍSTICOS   
# Analizamos la distribución de la cantidad de calificaciones por anime
counts_per_anime = rating_df['anime_id'].value_counts()
print(counts_per_anime.describe())
# Calcular e imprimir percentiles específicos
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_anime.quantile(p/100)))
# Analizamos la distribución de la cantidad de calificaciones por usuario
counts_per_user = rating_df['user_id'].value_counts()
print(counts_per_user.describe())
# Calcular e imprimir percentiles específicos
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_user.quantile(p/100)))

count    1990.000000
mean       58.000000
std        48.107722
min         3.000000
25%        24.000000
50%        42.000000
75%        78.750000
max       390.000000
Name: count, dtype: float64
P50 = 42
P75 = 78
P90 = 123
P95 = 163
P99 = 216
count    24364.000000
mean         4.737317
std          4.297914
min          1.000000
25%          2.000000
50%          3.000000
75%          6.000000
max         35.000000
Name: count, dtype: float64
P50 = 3
P75 = 6
P90 = 11
P95 = 13
P99 = 20


In [ ]:
# PASO 7 CREACION DE LA MATRIZ DE INTERACCION USUARIO-ANIME
anime_ratings = rating_df.pivot_table(index='user_id', columns='anime_id', values='rating', fill_value=0)
print(rating_df.head())

# PASO 8 CREAR LAS RECOMENDACIONES DE ANIME HUNTER X HUNTER CON ID 11061
# Aqui estan los Id de los animes con su recomendacion. En ese orden
TestRatings = pd.Series({11061: 10, 2476: 1})
# Suponiendo que 'rating_df' tiene columnas 'user_id', 'anime_id' y 'rating'





      user_id  anime_id  rating
1284       12       302       9
2254       19      8142      10
2265       20       819      10
2269       20      5391       9
2274       20      7739      10


,similarity,name
anime_id,,
8,-0.000945,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...
17,-0.001717,Mushishi Zoku Shou 2nd Season
18,-0.002642,Ookami Kodomo no Ame to Yuki
26,-0.002254,Monogatari Series: Second Season
28,-0.002641,Mushishi
49,-0.002763,Boku dake ga Inai Machi
55,-0.001036,Tengen Toppa Gurren Lagann Movie: Lagann-hen
56,-0.000966,Zoku Natsume Yuujinchou
58,-0.002781,Kuroko no Basket 3rd Season
